## Electronic markets project - exo1_1 and 1_2

### Loading libraries and initializing parameters

In [177]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

In [178]:
x0 = 10e4
sigma = 0.20
T = 1
eta = 1.1 * 1e-4
lambdas = [1e-3, 1e-2, 1e-1 , 1, 3, 8]

### Creating optimal strategy functions

In [179]:
def implementation_shortfall_opti_strat(t, x0, sigma, lamb, eta, T):
    """
    Computes implementation shortfall optimal deterministic strategy at a given time and for given parameters
    
    t: current time
    x0: initial inventory
    sigma: underlying spot volatility
    lamb: risk aversion parameter
    eta: multiplicative factor for temporary market impact
    T: end date of the execution process
    """
    
    k = sigma * np.sqrt(lamb / eta)
    return (x0 / np.sinh(k * T)) * np.sinh(k * (T - t))

In [180]:
def target_close_opti_strat(t, x0, sigma, lamb, eta, T):
    """
    Computes implementation shortfall optimal deterministic strategy at a given time and for given parameters
    
    t: current time
    x0: initial inventory
    sigma: underlying spot volatility
    lamb: risk aversion parameter
    eta: multiplicative factor for temporary market impact
    T: end date of the execution process
    """
    
    k = sigma * np.sqrt(lamb / eta)
    return x0 * (1 - np.sinh(k * t) / np.sinh(k * T))

In [181]:
def twap_opti_strat(t, x0, sigma, lamb, eta, T):
    """
    Computes implementation shortfall optimal deterministic strategy at a given time and for given parameters
    
    t: current time
    x0: initial inventory
    sigma: underlying spot volatility
    lamb: risk aversion parameter
    eta: multiplicative factor for temporary market impact
    T: end date of the execution process
    """
    
    k = sigma * np.sqrt(lamb / eta)
    return (x0 / np.sinh(k * T)) * np.sinh(k * (T - t))

### Implementation Shortfall - optimal strategy

#### Varying risk aversion

In [182]:
df = pd.DataFrame()

t_values = np.linspace(0, T, 1000)

for lamb in lambdas:
    xt = [implementation_shortfall_opti_strat(t, x0, sigma, lamb, eta, T) for t in t_values]
    temp_df = pd.DataFrame({'t': t_values, 'x': xt, 'lambda': str(lamb)})
    df = pd.concat([df, temp_df], ignore_index=True)

fig = px.line(df, x='t', y='x', color='lambda', template='plotly_white', 
              labels={"x": "Remaining Inventory", "t": "Time", "lambda": "Risk Aversion"})

fig.update_traces(hoverinfo='skip')
fig.update_traces(hovertemplate=None)
fig.update_layout(width=900, height=300)
fig.show()

#### Varying temporary market impact

In [65]:
df = pd.DataFrame()

t_values = np.linspace(0, T, 1000)
etas = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2]
lamb = 0.1

for eta in etas:
    xt = [implementation_shortfall_opti_strat(t, x0, sigma, lamb, eta, T) for t in t_values]
    temp_df = pd.DataFrame({'t': t_values, 'x': xt, 'eta': str(eta)})
    df = pd.concat([df, temp_df], ignore_index=True)

fig = px.line(df, x='t', y='x', color='eta', template='plotly_white', 
              labels={"x": "Remaining Inventory", "t": "Time", "eta": "Market Impact"})

fig.update_traces(hoverinfo='skip')
fig.update_traces(hovertemplate=None)
fig.update_layout(width=900, height=300)
fig.show()

### Target Close - optimal strategy

#### Varying risk aversion

In [76]:
df = pd.DataFrame()

t_values = np.linspace(0, T, 1000)

for lamb in lambdas:
    xt = [target_close_opti_strat(t, x0, sigma, lamb, eta, T) for t in t_values]
    temp_df = pd.DataFrame({'t': t_values, 'x': xt, 'lambda': str(lamb)})
    df = pd.concat([df, temp_df], ignore_index=True)

fig = px.line(df, x='t', y='x', color='lambda', template='plotly_white', 
              labels={"x": "Remaining Inventory", "t": "Time", "lambda": "Risk Aversion"})

fig.update_traces(hoverinfo='skip')
fig.update_traces(hovertemplate=None)
fig.update_layout(width=900, height=300)
fig.show()

#### Varying temporary market impact

In [77]:
df = pd.DataFrame()

t_values = np.linspace(0, T, 1000)
etas = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2]
lamb = 0.1

for eta in etas:
    xt = [target_close_opti_strat(t, x0, sigma, lamb, eta, T) for t in t_values]
    temp_df = pd.DataFrame({'t': t_values, 'x': xt, 'eta': str(eta)})
    df = pd.concat([df, temp_df], ignore_index=True)

fig = px.line(df, x='t', y='x', color='eta', template='plotly_white', 
              labels={"x": "Remaining Inventory", "t": "Time", "eta": "Market Impact"})

fig.update_traces(hoverinfo='skip')
fig.update_traces(hovertemplate=None)
fig.update_layout(width=900, height=300)
fig.show()

### TWAP - optimal strategy

#### Unique plot

In [80]:
df = pd.DataFrame()

t_values = np.linspace(0, T, 1000)
lamb = 0.1
    
xt = [twap_opti_strat(t, x0, sigma, lamb, eta, T) for t in t_values]
temp_df = pd.DataFrame({'t': t_values, 'x': xt, 'lambda': str(lamb)})
df = pd.concat([df, temp_df], ignore_index=True)

fig = px.line(df, x='t', y='x', color='lambda', template='plotly_white', 
              labels={"x": "Remaining Inventory", "t": "Time", "lambda": "Risk Aversion"})

fig.update_traces(hoverinfo='skip')
fig.update_traces(hovertemplate=None)
fig.update_layout(width=900, height=300)
fig.show()

### Implementation Shortfall - efficient frontier

In [187]:
def calculate_IS_expectation(k, eta, sigma, x0, T):
    return -1 * ((((k * x0) / (np.sinh(k * T)))**2) * eta) * ((2 * k * T + np.sinh(2 * k * T)) / (4 * k))

def calculate_IS_variance(k, sigma, x0, T):
    return (((sigma * x0) / (np.sinh(k * T)))**2) * ((np.sinh(2 * k * T) - (2 * k * T)) / (4 * k))

lbd_values = np.linspace(1e-4, 1, num=1000)
k_values = np.sqrt(lbd_values / eta) * sigma

expectation = calculate_IS_expectation(k_values, eta, sigma, x0, T)
variance = calculate_IS_variance(k_values, sigma, x0, T)

df = pd.DataFrame({'E': np.abs(expectation) / 100, 'V': np.abs(variance) / 10000, 'lambda': lbd_values})

fig = px.scatter(df, x='V', y='E', color='lambda', template='plotly_white', 
                 labels={"V": "Variance", "E": "Abs(Expectation)", "lambda": "𝛌"}, color_continuous_scale='Purp')
fig.update_layout(width=900, height=300)
fig.show()

### Target Close - efficient frontier

In [186]:
def calculate_TC_expectation(k, eta, sigma, x0, T):
    return -1 * ((((k * x0) / (np.sinh(k * T)))**2) * eta) * ((2 * k * T + np.sinh(2 * k * T)) / (4 * k))

def calculate_TC_variance(k, sigma, x0, T):
    return (((sigma * x0) / (np.sinh(k * T)))**2) * ((np.sinh(2 * k * T) - (2 * k * T)) / (4 * k))

lbd_values = np.linspace(1e-4, 1, num=1000)
k_values = np.sqrt(lbd_values / eta) * sigma

expectation = calculate_IS_expectation(k_values, eta, sigma, x0, T)
variance = calculate_IS_variance(k_values, sigma, x0, T)

df = pd.DataFrame({'E': np.abs(expectation) / 100, 'V': np.abs(variance) / 10000, 'lambda': lbd_values})

fig = px.scatter(df, x='V', y='E', color='lambda', template='plotly_white', 
                 labels={"V": "Variance", "E": "Abs(Expectation)", "lambda": "𝛌"}, color_continuous_scale='Purp')
fig.update_layout(width=900, height=300)
fig.show()